In [ ]:
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
import lsst.summit.utils.butlerUtils as butlerUtils

client = makeEfdClient()
butler = butlerUtils.makeDefaultButler(instrument='LSSTComCam')

expId = 2024121100610
dataId = {'exposure': expId, 'detector': 4, 'instrument': 'LSSTComCam'}
expRecord = butlerUtils.getExpRecordFromDataId(butler, dataId)
rotData = getEfdData(client=client, topic="lsst.sal.MTRotator.rotation", expRecord=expRecord)
pos = rotData['actualPosition'].values.mean()
print(f"For {expId}, mean rotator position was {pos:.2f} degrees")

In [ ]:
from lsst.summit.utils.efdUtils import calcNextDay
import matplotlib.pyplot as plt

startDay = 20241022
endDay = 20241211
dayObs = startDay

rots = []
while dayObs <= endDay:
    records = butler.registry.queryDimensionRecords("exposure", where="exposure.day_obs=%d"%dayObs)
    if records.count() < 100:
        dayObs = calcNextDay(dayObs)  # find the next dayObs                                                                                       
        continue
    exposureList = []
    for record in butler.registry.queryDimensionRecords("exposure", where="exposure.day_obs=%d"%dayObs):
        if (record.observation_type=='science'):
            exposureList.append(record.id)
    exposureList = sorted(exposureList)
    print(dayObs, len(exposureList))
    for expId in exposureList:
        try:
            dataId = {'exposure': expId, 'detector': 4, 'instrument': 'LSSTComCam'}
            expRecord = butlerUtils.getExpRecordFromDataId(butler, dataId)
            rotData = getEfdData(client=client, topic="lsst.sal.MTRotator.rotation", expRecord=expRecord)
            pos = rotData['actualPosition'].values.mean()
            rots.append(pos)
        except:
            continue
    dayObs = calcNextDay(dayObs)

plt.hist(rots, bins=50)
plt.xlim(-90,90)
plt.title("ComCam campaign Rotator angle of science images")
plt.xlabel("Rotator physical angle (degrees)")

In [ ]:
plt.hist(rots, bins=50)
plt.xlim(-90,90)
plt.title("ComCam campaign Rotator angle of science images")
plt.xlabel("Rotator physical angle (degrees)")
plt.savefig("plots/Rotator_Histogram_ComCam.png")
